In [ ]:
!pip install requests beautifulsoup4


In [ ]:
!pip install playwright
!playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 10.5 MB/s eta 0:00:00
167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 21.4s167.7 MiB [] 0% 27.0s167.7 MiB [] 0% 11.3s167.7 MiB [] 0% 8.6s167.7 MiB [] 1% 7.6s167.7 MiB [] 1% 7.2s167.7 MiB [] 1% 6.6s167.7 MiB [] 2% 6.0s167.7 MiB [] 2% 5.7s167.7 MiB [] 3% 5.3s167.7 MiB [] 3% 5.0s167.7 MiB [] 4% 4.7s167.7 MiB [] 4% 4.6s167.7 MiB [] 4% 4.5s167.7 MiB [] 5% 4.6s167.7 MiB [] 5% 5.0s167.7 MiB [] 5% 5.1s167.7 MiB [] 6% 5.1s167.7 MiB [] 6% 5.2s167.7 MiB [] 6% 5.0s167.7 MiB [] 7% 4.8s167.7 MiB [] 7% 4.9s167.7 MiB [] 7% 5.0s167.7 MiB [] 8% 4.8s167.7 MiB [] 8% 4.7s167.7 MiB [] 9% 4.8s167.7 MiB [] 9% 4.6s167.7 MiB [] 10% 4.5s167.7 MiB [] 10% 4.4s167.7 MiB [] 10% 4.5s167.7 MiB [] 11% 4.4s167.7 MiB [] 11% 4.3s167.7 MiB [] 12% 4.1s167.7 MiB [] 13% 4.0s167.7 MiB [] 14% 3.8s167.7 MiB [] 14% 3.7s167.7 MiB [] 15% 3.6s167.7 MiB [] 15% 3.7s167.7 MiB [] 16% 3.6s167.7 MiB [] 17% 3.4s167.7 MiB [] 17% 3.3s167.7 MiB [] 18% 3.3s167.7 MiB [] 18% 3.4s167.7 MiB [] 18% 3.3s1

## Use the code to now scrape 10 articles and perform sentiment analysis

In [ ]:
import nest_asyncio
import asyncio
import json
import pandas as pd
import numpy as np
import re
import datetime
from playwright.async_api import async_playwright
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Apply nested asyncio to allow await in Jupyter
nest_asyncio.apply()


In [ ]:
!pip install python-dotenv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1️⃣ Extract relevant text containing keywords
# def extract_relevant_text(text, keywords):
#     sentences = re.split(r'(?<=[.!?])\s+', text)
#     relevant_sentences = [
#         sentence for sentence in sentences
#         if any(keyword.lower() in sentence.lower() for keyword in keywords)
#     ]
#     return " ".join(relevant_sentences) if relevant_sentences else text

In [ ]:
def extract_relevant_paragraphs(text, ticker, company_name):
    """
    Extract paragraphs containing the specified ticker or company name.

    Args:
        text (str): The article text.
        ticker (str): The stock ticker symbol (e.g., 'HD').
        company_name (str): The company name (e.g., 'Home Depot').

    Returns:
        str: Combined text of relevant paragraphs.
    """
    paragraphs = text.split('\n\n')  # split text into paragraphs
    relevant = []
    for para in paragraphs:
        if ticker.upper() in para.upper() or company_name.lower() in para.lower():
            relevant.append(para)
    return '\n\n'.join(relevant)

In [ ]:
async def fetch_article_links(ticker, limit=15):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        url = f"https://finance.yahoo.com/quote/{ticker}/news"
        try:
            await page.goto(url, timeout=60000)
            print(f"✅ Navigated to {url}")
        except Exception as e:
            print(f"⚠️ Failed to navigate to {url}: {e}")
            await browser.close()
            return []

        # Consent
        try:
            await page.click('button[name="agree"]')
            print("✅ Clicked consent button.")
        except:
            print("⚠️ No consent button found. Continuing.")

        try:
            await page.wait_for_selector("a.subtle-link", timeout=15000)
            links = await page.eval_on_selector_all(
                "a.subtle-link",
                "els => els.map(el => el.href)"
            )
            # Deduplicate
            unique_links = list(dict.fromkeys(links))

            # Filter only /news/ articles
            article_links = [link for link in unique_links if '/news/' in link]

            # Further optional filter: contains the ticker symbol
            # article_links = [link for link in article_links if ticker.lower() in link.lower()]

            limited_links = article_links[:limit]
            print(f"✅ Found {len(limited_links)} unique article links for {ticker}.")
            await browser.close()
            return limited_links
        except Exception as e:
            print(f"⚠️ No links found or error occurred: {e}")
            await browser.close()
            return []


In [ ]:
#testing link retrieval
links = await fetch_article_links("AMZN")
print(links)



✅ Navigated to https://finance.yahoo.com/quote/AMZN/news
⚠️ No consent button found. Continuing.
✅ Found 15 unique article links for AMZN.
['https://finance.yahoo.com/news/warren-buffetts-ai-bets-22-113000614.html', 'https://finance.yahoo.com/news/palantir-still-buy-run-3-110000432.html', 'https://finance.yahoo.com/news/amazon-eyes-ai-powered-humanoid-100418326.html', 'https://finance.yahoo.com/news/1-magnificent-p-500-dividend-093100956.html', 'https://finance.yahoo.com/news/3-best-growth-stocks-buy-083000594.html', 'https://finance.yahoo.com/news/prediction-2-ai-stocks-worth-081000147.html', 'https://finance.yahoo.com/news/too-buy-mercadolibre-stock-074100790.html', 'https://finance.yahoo.com/news/meta-talks-scale-ai-investment-053458408.html', 'https://finance.yahoo.com/news/amazon-com-nasdaqgs-amzn-expands-171222318.html', 'https://finance.yahoo.com/news/could-only-buy-hold-single-144700512.html', 'https://finance.yahoo.com/news/2-no-brainer-stocks-id-104500257.html', 'https://fina

In [ ]:

# 🔹 Fetch Article Details

import datetime
from dateutil import parser

async def fetch_article_text(link):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto(link, wait_until='domcontentloaded', timeout=90000)
        except Exception as e:
            print(f"⚠️ Failed to navigate to {link}: {e}")
            await browser.close()
            return {"title": "", "text": "", "pub_date": None}

        try:
            title = await page.title()
            # fallback selectors
            try:
                text = await page.locator("div.caas-body").inner_text(timeout=5000)
            except:
                text = await page.locator("article").inner_text(timeout=5000)

            # fallback date (JSON-LD can be parsed if needed)
            # Parse publication date
            pub_date = None
            try:
                time_element = await page.locator("time").first.get_attribute("datetime")
                if time_element:
                    dt = parser.parse(time_element)
                    pub_date = dt.date().isoformat()
            except Exception as e:
                print(f"⚠️ Failed to parse date for {link}: {e}")

        except Exception as e:
            print(f"Error fetching article {link}: {e}")
            title = ""
            text = ""
            pub_date = None

        await browser.close()
        return {"title": title, "text": text, "pub_date": pub_date}



## Sentiment analysis

In [ ]:
# Load FinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

def analyze_sentiment(text, ticker, name):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    paragraph_scores = []

    for paragraph in paragraphs:
        inputs = tokenizer(paragraph, return_tensors='pt', truncation=True, max_length=512)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        probs_np = probs.detach().numpy()[0]
        single_score = (probs_np[0] * 0) + (probs_np[1] * 0.5) + (probs_np[2] * 1)
        paragraph_scores.append(single_score)

    avg_score = np.mean(paragraph_scores)
    if np.isnan(avg_score):
        avg_score = 0.5  # fallback to neutral sentiment
    avg_score = round(avg_score, 3)

    if avg_score >= 0.6:
        overall_sentiment = "positive"
    elif avg_score < 0.4:
        overall_sentiment = "negative"
    else:
        overall_sentiment = "neutral"

    return overall_sentiment, avg_score





In [ ]:
# 🔹 Main Pipeline Function
async def run_pipeline(ticker, company_name, limit=15, target_days=5):
    links = await fetch_article_links(ticker, limit=limit)
    print(f"✅ Found {len(links)} unique article links.")

    articles = []
    for link in links:
        print(f"\n🔗 Scraping Article: {link}")
        article = await fetch_article_text(link)
        if article["text"]:
            relevant_text = extract_relevant_paragraphs(article['text'], ticker, company_name)
            sentiment, score = analyze_sentiment(relevant_text, ticker, company_name)
            print(f"✅ Article added with sentiment: {sentiment} (Score: {score:.3f})")
            articles.append({
                "ticker": ticker,
                "title": article["title"],
                "date": article["pub_date"],
                "sentiment": sentiment,
                "sentiment_score": score  # Corrected key
            })

    df = pd.DataFrame(articles)
    print(f"\n✅ Raw DataFrame shape: {df.shape}")
    print(df.head())

    # Aggregate by date
    aggregated_df = (
        df.groupby(['ticker', 'date'])
        .agg({
            'sentiment_score': 'mean',
            'sentiment': lambda x: ','.join(x),
            'title': lambda x: ' | '.join(x)
        })
        .reset_index()
    )

    print(f"✅ Aggregated DataFrame shape: {aggregated_df.shape}")

    # Sort and get latest N days
    aggregated_df = aggregated_df.sort_values(['ticker', 'date'], ascending=[True, False])
    latest_days_df = aggregated_df.groupby('ticker').head(target_days)

    print(f"✅ Filtered to latest {target_days} days:")
    print(latest_days_df)

    # Forward-fill sentiment (optional, typically done during merge with stock prices)
    latest_days_df = latest_days_df.sort_values(['ticker', 'date'])
    latest_days_df['sentiment_score'] = (
        latest_days_df.groupby('ticker')['sentiment_score']
        .ffill()
        .fillna(0.5)
        .round(3)
    )

    print(f"✅ Forward-filled DataFrame:")
    print(latest_days_df)

    return latest_days_df


In [ ]:
# Example: Run for ticker "HD"
df = await run_pipeline("HD", "Home Depot")

✅ Navigated to https://finance.yahoo.com/quote/HD/news
⚠️ No consent button found. Continuing.
✅ Found 15 unique article links.
✅ Found 15 unique article links.

🔗 Scraping Article: https://finance.yahoo.com/news/best-stock-buy-now-costco-101500247.html
✅ Article added with sentiment: neutral (Score: 0.6949999928474426)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-inc-hd-attracting-130015318.html
✅ Article added with sentiment: negative (Score: 0.11599999666213989)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-bets-pros-again-170400878.html
✅ Article added with sentiment: neutral (Score: 0.33399999141693115)

🔗 Scraping Article: https://finance.yahoo.com/news/1-cash-producing-stock-exciting-043310050.html
✅ Article added with sentiment: negative (Score: 0.19599999487400055)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-stocks-high-p-144900654.html
✅ Article added with sentiment: neutral (Score: 0.48399999737739563)

🔗 Scraping Article

CancelledError: 

## Run sentiment pipeline

In [ ]:
pd.set_option('display.float_format', '{:.3f}'.format)


# Load the consumer discretionary CSV
consumer_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MSDS459_Final_Project/consumer_discretionary_sites.csv')

all_articles = []

# Parameters
LIMIT_ARTICLES = 30   # How many articles to fetch initially per ticker
TARGET_DAYS = 30     # How many days to keep per ticker

for idx, row in consumer_df.iterrows():
    ticker = row['symbol']
    company_name = row['name']
    print(f"🔍 Processing Item {idx}: {ticker} ({company_name})")

    # Use positional arguments since run_pipeline expects them in that order
    df = await run_pipeline(
        ticker,
        company_name,
        LIMIT_ARTICLES,
        TARGET_DAYS
    )
    all_articles.append(df)

final_df = pd.concat(all_articles, ignore_index=True)
print("✅ Final Sentiment DataFrame:")
print(final_df.head())

🔍 Processing AMZN (Amazon.com, Inc.)
✅ Navigated to https://finance.yahoo.com/quote/AMZN/news
⚠️ No consent button found. Continuing.
✅ Found 18 unique article links for AMZN.
✅ Found 18 unique article links.

🔗 Scraping Article: https://finance.yahoo.com/news/warren-buffetts-ai-bets-22-113000614.html
✅ Article added with sentiment: negative (Score: 0.000)

🔗 Scraping Article: https://finance.yahoo.com/news/palantir-still-buy-run-3-110000432.html
✅ Article added with sentiment: negative (Score: 0.110)

🔗 Scraping Article: https://finance.yahoo.com/news/amazon-eyes-ai-powered-humanoid-100418326.html
✅ Article added with sentiment: negative (Score: 0.243)

🔗 Scraping Article: https://finance.yahoo.com/news/1-magnificent-p-500-dividend-093100956.html
✅ Article added with sentiment: positive (Score: 0.725)

🔗 Scraping Article: https://finance.yahoo.com/news/3-best-growth-stocks-buy-083000594.html
✅ Article added with sentiment: negative (Score: 0.183)

🔗 Scraping Article: https://finance.y

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


✅ Article added with sentiment: neutral (Score: 0.500)

✅ Raw DataFrame shape: (18, 5)
  ticker                                              title        date  \
0   TSLA  A fresh inflation reading greets a stock marke...  2025-06-08   
1   TSLA  Retail Investors Pile Into Tesla (TSLA) After ...  2025-06-08   
2   TSLA         Does the EV Industry Have a Truck Problem?  2025-06-08   
3   TSLA  Veteran fund manager revamps stock market fore...  2025-06-07   
4   TSLA  Vance Says He Hopes Musk Returns to Fold After...  2025-06-07   

  sentiment  sentiment_score  
0  negative            0.000  
1  negative            0.360  
2  negative            0.399  
3   neutral            0.495  
4  negative            0.000  
✅ Aggregated DataFrame shape: (3, 5)
✅ Filtered to latest 30 days:
  ticker        date  sentiment_score  \
2   TSLA  2025-06-08            0.253   
1   TSLA  2025-06-07            0.226   
0   TSLA  2025-06-06            0.340   

                                           s

ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


⚠️ No consent button found. Continuing.
✅ Found 21 unique article links for HD.
✅ Found 21 unique article links.

🔗 Scraping Article: https://finance.yahoo.com/news/best-stock-buy-now-costco-101500247.html
✅ Article added with sentiment: neutral (Score: 0.519)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-inc-hd-attracting-130015318.html
✅ Article added with sentiment: negative (Score: 0.010)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-bets-pros-again-170400878.html
✅ Article added with sentiment: negative (Score: 0.335)

🔗 Scraping Article: https://finance.yahoo.com/news/1-cash-producing-stock-exciting-043310050.html
✅ Article added with sentiment: negative (Score: 0.200)

🔗 Scraping Article: https://finance.yahoo.com/news/home-depot-stocks-high-p-144900654.html
✅ Article added with sentiment: neutral (Score: 0.430)

🔗 Scraping Article: https://finance.yahoo.com/news/3-rock-solid-dow-jones-111500450.html
✅ Article added with sentiment: neutral (Scor

...

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


✅ Article added with sentiment: neutral (Score: 0.500)

✅ Raw DataFrame shape: (17, 5)
  ticker                                              title        date  \
0   SBUX  Starbucks makes organisational changes and nam...  2025-06-06   
1   SBUX                     Starbucks revives COO position  2025-06-05   
2   SBUX               3 Nasdaq 100 Stocks Facing Headwinds  2025-06-05   
3   SBUX  Starbucks (SBUX) Surpasses Market Returns: Som...  2025-06-04   
4   SBUX  Starbucks Reshuffles Leadership to Accelerate ...  2025-06-04   

  sentiment  sentiment_score  
0  negative            0.000  
1  negative            0.000  
2  negative            0.173  
3  negative            0.333  
4  negative            0.211  
✅ Aggregated DataFrame shape: (7, 5)
✅ Filtered to latest 30 days:
  ticker        date  sentiment_score  \
6   SBUX  2025-06-06            0.000   
5   SBUX  2025-06-05            0.086   
4   SBUX  2025-06-04            0.159   
3   SBUX  2025-05-31            0.452   
2   

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


✅ Article added with sentiment: neutral (Score: 0.500)

✅ Raw DataFrame shape: (20, 5)
  ticker                                              title        date  \
0   ORLY  Is O'Reilly Automotive Stock a Buy After Its 1...  2025-06-06   
1   ORLY  Tariffs Aren't O'Reilly's Biggest Problem, Thi...  2025-06-04   
2   ORLY  Auto Parts Retailer Stocks Q1 Earnings: Advanc...  2025-06-03   
3   ORLY  Wall Street's Biggest Stock-Split Stock of 202...  2025-06-02   
4   ORLY  Could Investing $10,000 in O'Reilly Automotive...  2025-05-28   

  sentiment  sentiment_score  
0   neutral            0.500  
1  negative            0.263  
2  negative            0.000  
3   neutral            0.500  
4  negative            0.333  
✅ Aggregated DataFrame shape: (15, 5)
✅ Filtered to latest 30 days:
   ticker        date  sentiment_score                   sentiment  \
14   ORLY  2025-06-06            0.500                     neutral   
13   ORLY  2025-06-04            0.263                    negative  

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
final_df.to_csv('/content/drive/My Drive/Colab Notebooks/MSDS459_Final_Project/sentiment_results_bert.csv', index=False)

In [ ]:
final_df.head()

,ticker,date,sentiment_score,sentiment,title
0,AMZN,2025-06-07,0.112,"negative,negative,negative,negative,negative,n...",Amazon.com (NasdaqGS:AMZN) Expands AWS Presenc...
1,AMZN,2025-06-08,0.189,"negative,negative,negative,positive,negative,n...",Warren Buffett's AI Bets: 22% of Berkshire Hat...
2,TSLA,2025-06-06,0.340,negative,Departure of Tesla’s humanoid robot boss caps ...
3,TSLA,2025-06-07,0.226,"neutral,negative,negative,negative,negative,ne...",Veteran fund manager revamps stock market fore...
4,TSLA,2025-06-08,0.253,"negative,negative,negative",A fresh inflation reading greets a stock marke...
